In [32]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from collections import Counter
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, fbeta_score

In [33]:
# daten laden / teilen
df = pd.read_csv('../data/GeneralDatensatz18-21ohneGeo-mitLockdown_mitCorona.csv', sep=';')

X=df[['UMONAT','USTUNDE','UWOCHENTAG','UART','USTRZUSTAND','BEZ','UTYP1','ULICHTVERH','IstRad','IstPKW','IstFuss','IstKrad','IstGkfz','IstSonstige', 'LOCKDOWN', 'COVID']]
#für tödliche Unfälle

#für tödliche und schwere vs. leichte Unfälle  -> 1 ist schwer oder tödlich, 0 ist leicht
y = df['UKATEGORIE'].isin([1, 2]).astype(int)


In [34]:
# StratifiedKFold initialisieren
skf = StratifiedKFold(n_splits=5)

# Variablen für Ergebnisse
fbetas_rf = []
fbetas_random = []
fbetas_mehrheit = []

In [35]:
beta = 2
fbeta_scorer= make_scorer(fbeta_score, beta=beta)

In [36]:
from sklearn.model_selection import cross_val_score

counter = 0 
# Stratified K-Fold Cross-Validation
for (train_index, test_index) in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    
    # Random Forest Modell trainieren
    model = RandomForestClassifier(n_estimators=344, min_samples_split= 400, min_samples_leaf= 8, max_features ='log2', max_depth= 15, class_weight = {0: 1, 1: 9}, random_state=42)
    
    model.fit(X_train, y_train)
    
    # Vorhersagen mit dem Random Forest Modell
    y_pred_rf = model.predict(X_test)
    print("ersten 10 Vorhersagen mit dem Random Forest Modelly", y_pred_rf[:10])
   
    
    # Leistung des Random Forest Modells bewerten
    fbeta_rf = fbeta_score(y_test, y_pred_rf, beta=beta, average='weighted')
    print(f"Fold nummer {counter}: fbeta - score ist {fbeta_rf} ")
    fbetas_rf.append(fbeta_rf)
    
     # jetzt mit random:

    # Klassenverteilung im Testdatensatz ermitteln
    class_counts = Counter(y_test)
    total_samples = len(y_test)
    class_probabilities = {cls: count / total_samples for cls, count in class_counts.items()}
    
    # Zufällige Vorhersagen basierend auf den Klassenwahrscheinlichkeiten erstellen
    np.random.seed(42)
    y_pred_random_weighted = np.random.choice(
        list(class_probabilities.keys()),
        size=y_test.shape,
        p=list(class_probabilities.values())
    )
    #drucke die ersten 10 Werte
    print("ersten 10 Vorhersagen mit dem _random_weighted",y_pred_random_weighted[:10])
    
    # Leistung der gewichteten zufälligen Vorhersagen bewerten
    fbeta_random_weighted = fbeta_score(y_test, y_pred_random_weighted, beta=beta, average='weighted')
    print(f"Fold nummer {counter}: fbeta_random_weighted - score ist {fbeta_random_weighted} ")
    fbetas_random.append(fbeta_random_weighted)
    
    # immer mehrheitsklasse vorhersagen
    y_pred_zeros =np.zeros(y_test.shape)
   
     #drucke die ersten 10 Werte
    print("ersten 10 Vorhersagen mit dem y_pred_zeros",y_pred_zeros[:10])
    
    # Leistung der Vorhersagen der mehrheitsklasse bewerten
    fbeta_mehrheit = fbeta_score(y_test, y_pred_zeros, beta=beta, average='weighted')
    print(f"Fold nummer {counter}: fbeta_mehrheit - score ist {fbeta_mehrheit} ")
    fbetas_mehrheit.append(fbeta_mehrheit)
    counter = counter+1
    
    

ersten 10 Vorhersagen mit dem Random Forest Modelly [1 1 1 1 0 0 1 1 0 1]
Fold nummer 0: fbeta - score ist 0.40011381206946167 
ersten 10 Vorhersagen mit dem _random_weighted [0 1 0 0 0 0 0 1 0 0]
Fold nummer 0: fbeta_random_weighted - score ist 0.7418018382188678 
ersten 10 Vorhersagen mit dem y_pred_zeros [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Fold nummer 0: fbeta_mehrheit - score ist 0.8166006394009186 
ersten 10 Vorhersagen mit dem Random Forest Modelly [0 0 1 1 1 1 1 0 1 0]
Fold nummer 1: fbeta - score ist 0.3841144891675201 
ersten 10 Vorhersagen mit dem _random_weighted [0 1 0 0 0 0 0 1 0 0]
Fold nummer 1: fbeta_random_weighted - score ist 0.7418018382188678 
ersten 10 Vorhersagen mit dem y_pred_zeros [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Fold nummer 1: fbeta_mehrheit - score ist 0.8166006394009186 
ersten 10 Vorhersagen mit dem Random Forest Modelly [1 0 1 0 1 1 1 0 1 0]
Fold nummer 2: fbeta - score ist 0.40011689280332147 
ersten 10 Vorhersagen mit dem _random_weighted [0 1 0 0 0 0 0 1 0 0

In [37]:
# Durchschnittliche Genauigkeiten berechnen
mean_scores_rf = np.mean(fbetas_rf)
mean_scores_random = np.mean(fbetas_random)
mean_scores_mehrheit = np.mean(fbetas_mehrheit)

In [38]:
print(f"LogReg Mean score: {mean_scores_rf}")
print(f"Weighted Random Prediction Mean score: {mean_scores_random}")
print(f"Weighted Mehrheit Prediction Mean score: {mean_scores_mehrheit}")


LogReg Mean score: 0.3900273895954972
Weighted Random Prediction Mean score: 0.7429524231216044
Weighted Mehrheit Prediction Mean score: 0.8165933693152164


In [39]:

# Klassifikationsberichte für den letzten Fold ausgeben
print("LogReg Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_rf))

print("Weighted Random Prediction Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_random_weighted))

LogReg Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.94      0.30      0.45      8482
           1       0.19      0.89      0.31      1541

    accuracy                           0.39     10023
   macro avg       0.56      0.59      0.38     10023
weighted avg       0.82      0.39      0.43     10023

Weighted Random Prediction Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      8482
           1       0.15      0.14      0.14      1541

    accuracy                           0.74     10023
   macro avg       0.50      0.50      0.50     10023
weighted avg       0.74      0.74      0.74     10023

